In [1]:
from moccasin import setup_notebook

setup_notebook()

In [2]:
from moccasin.config import get_active_network

active_network = get_active_network()
print(f"Active network: {active_network.name}")

Active network: eth-forked


In [3]:
import boa
from boa.contracts.abi.abi_contract import ABIContract
from moccasin.config import get_active_network, Network

STARTING_ETH_BALANCE = int(1000e18)  # 1000 ETH
STARTING_WETH_BALANCE = int(1e18)  # 1 wETH
STARTING_USDC_BALANCE = int(100e6)  # 100 USDC (6 decimals)


def _add_eth_balance() -> None:
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)


def _add_token_balance(usdc: ABIContract, weth: ABIContract) -> None:
    weth.deposit(value=STARTING_WETH_BALANCE)

    our_address = boa.env.eoa
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
    usdc.configureMinter(our_address, STARTING_USDC_BALANCE)
    usdc.mint(our_address, STARTING_USDC_BALANCE)


def setup_script() -> (ABIContract, ABIContract, ABIContract, ABIContract):
    print("Starting setup script...")

    # Give ourselves some ETH

    # Give ourselves some USDC and wETH
    active_network = get_active_network()

    usdc = active_network.manifest_named("usdc")
    weth = active_network.manifest_named("weth")

    if active_network.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc, weth)

    return (usdc, weth)


def moccasin_main():
    (usdc, weth) = setup_script()
    return (usdc, weth)


moccasin_main()

Starting setup script...


(<usdc interface at 0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48>,
 <weth interface at 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2>)

In [4]:
usdc = active_network.manifest_named("usdc")
weth = active_network.manifest_named("weth")

In [5]:
from moccasin.config import get_config

config = get_config()
config.reload()
active_network = get_active_network()

In [6]:
aave_pool_address_provider = active_network.manifest_named("aave_pool_address_provider")
pool_address = aave_pool_address_provider.getPool()
print(f"Aave Pool address: {pool_address}")

Aave Pool address: 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2


In [7]:
config.reload()
active_network = get_active_network()
pool_contract = active_network.manifest_named("pool", address=pool_address)

In [8]:
REFERRAL_CODE = 0

def deposit(pool_contract, token, amount):
    allowed_amount = token.allowance(boa.env.eoa, pool_contract.address)
    if allowed_amount < amount:
        token.approve(pool_contract.address, amount)
    print(f"Depositing {amount} of token {token.name()} to Aave Pool {pool_contract.address}...")
    pool_contract.supply(token.address, amount, boa.env.eoa, REFERRAL_CODE)

usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)

if usdc_balance > 0:
    deposit(pool_contract, usdc, usdc_balance)

if weth_balance > 0:
    deposit(pool_contract, weth, weth_balance)

(
    totalCollateralBase,
    totalDebtBase,
    availableBorrowBase,
    currentLiquidationThreshold,
    ltv,
    healthFactor,
) = pool_contract.getUserAccountData(boa.env.eoa)
print(f"""User account data:
    totalCollateralBase: {totalCollateralBase}
    totalDebtBase: {totalDebtBase}
    availableBorrowBase: {availableBorrowBase}
    currentLiquidationThreshold: {currentLiquidationThreshold}
    ltv: {ltv}
    healthFactor: {healthFactor}
""")

Depositing 100000000 of token USD Coin to Aave Pool 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2...
Depositing 1000000000000000000 of token Wrapped Ether to Aave Pool 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2...
User account data:
    totalCollateralBase: 207650241799
    totalDebtBase: 0
    availableBorrowBase: 166597788995
    currentLiquidationThreshold: 8275
    ltv: 8023
    healthFactor: 115792089237316195423570985008687907853269984665640564039457584007913129639935



## We want 30% of our portfolio in USDC and 70% in WETH.

In [9]:
config.reload()
active_network = get_active_network()
aave_protocol_data_provider = active_network.manifest_named("aave_protocol_data_provider")
a_tokens = aave_protocol_data_provider.getAllATokens()
print(f"All aTokens: {a_tokens}")

All aTokens: [getAllATokens(symbol='aEthWETH', tokenAddress=Address('0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8')), getAllATokens(symbol='aEthwstETH', tokenAddress=Address('0x0B925eD163218f6662a35e0f0371Ac234f9E9371')), getAllATokens(symbol='aEthWBTC', tokenAddress=Address('0x5Ee5bf7ae06D1Be5997A1A72006FE6C607eC6DE8')), getAllATokens(symbol='aEthUSDC', tokenAddress=Address('0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c')), getAllATokens(symbol='aEthDAI', tokenAddress=Address('0x018008bfb33d285247A21d44E50697654f754e63')), getAllATokens(symbol='aEthLINK', tokenAddress=Address('0x5E8C8A7243651DB1384C0dDfDbE39761E8e7E51a')), getAllATokens(symbol='aEthAAVE', tokenAddress=Address('0xA700b4eB416Be35b2911fd5Dee80678ff64fF6C9')), getAllATokens(symbol='aEthcbETH', tokenAddress=Address('0x977b6fc5dE62598B08C85AC8Cf2b745874E8b78c')), getAllATokens(symbol='aEthUSDT', tokenAddress=Address('0x23878914EFE38d27C4D67Ab83ed1b93A74D4086a')), getAllATokens(symbol='aEthrETH', tokenAddress=Address('0xCc9EE9483

In [11]:
for a_token in a_tokens:
    if "WETH" in a_token[0]:
        a_weth = active_network.manifest_named("usdc", address = a_token[1])
    if "USDC" in a_token[0]:
        a_usdc = active_network.manifest_named("usdc", address = a_token[1])

print(a_usdc)
print(a_weth)

<usdc interface at 0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c>
<usdc interface at 0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8>


In [12]:
# Get how much they are worth, to figure out what our portfolio portions are in USD.
a_usdc_balance = a_usdc.balanceOf(boa.env.eoa) # 6 decimals
a_weth_balance = a_weth.balanceOf(boa.env.eoa) # 18 decimals

a_usdc_balance_normalized = a_usdc_balance / 1e6
a_weth_balance_normalized = a_weth_balance / 1e18
print(f"USDC balance: {a_usdc_balance_normalized}")
print(f"WETH balance: {a_weth_balance_normalized}")

USDC balance: 99.999998
WETH balance: 1.0


In [ ]:
# Chainlink ETH/USD price feed address: 0x5f4eC3Df9cbd43714FE2740f5E3616155c5b8419
# USDC/USD price feed address: 0x8fFfFfd4AfB6115b954Bd326cbe7B4BA576818f6

def get_price(feed_name) -> float:
    active_network = get_active_network()
    price_feed = active_network.manifest_named(feed_name)
    price = price_feed.latestAnswer()
    decimals = price_feed.decimals()
    normalized_price = price / (10 ** decimals)
    return normalized_price

eth_usd_price = get_price("eth_usd")
usdc_usd_price = get_price("usdc_usd")
print(f"ETH/USD price: {eth_usd_price}")
print(f"USDC/USD price: {usdc_usd_price}")